In [ ]:
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd
from dash.dependencies import Input, Output
import json
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.express as px
from dash.dependencies import Input, Output
import plotly.graph_objs as go
from dash.dependencies import Output, Input, State



url = "https://data.cityofnewyork.us/resource/qiz3-axqb.json"
collisions = pd.read_json(url)



SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#2b2b2b",
    "color": "#E8E8E8",
}


CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
 
}

#-----------------------------------Define  Boxs Here------------------------------------------------------


fig_box1 = px.bar(collisions, x = collisions['crash_date'], y = collisions['number_of_persons_injured'])
fig_box1.update_layout(yaxis={'visible': False, 'showticklabels': True},   paper_bgcolor='#1e1e1e',
    plot_bgcolor=' #2b2b2b', font= {"color": "#ffc434"},)

fig_box2 = px.bar(collisions, x = collisions['on_street_name'], y = collisions['number_of_persons_injured'])
fig_box2.update_layout(yaxis={'visible': False, 'showticklabels': True},   paper_bgcolor='#1e1e1e',
    plot_bgcolor=' #2b2b2b', font= {"color": "#ffc434"},)


#-----------------------------------Define 1. Map Here------------------------------------------------------

fig_map_1 = px.scatter_mapbox(collisions, lat="latitude", lon="longitude", hover_name="on_street_name", 
                              hover_data=["on_street_name", "number_of_persons_injured"],
                        color_discrete_sequence=["#ffc434"], zoom=7, height=300)
fig_map_1.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        },
        {
            "sourcetype": "raster",
            "sourceattribution": "Government of Canada",
            "source": ["https://geo.weather.gc.ca/geomet/?"
                       "SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX={bbox-epsg-3857}&CRS=EPSG:3857"
                       "&WIDTH=1000&HEIGHT=1000&LAYERS=RADAR_1KM_RDBR&TILED=true&FORMAT=image/png"],
        }
      ])
fig_map_1.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


#-----------------------------------Define 2. Map Here------------------------------------------------------


fig_map_2 = px.scatter_mapbox(collisions, lat="latitude", lon="longitude", hover_name="on_street_name", 
                              hover_data=["on_street_name", "number_of_persons_killed"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig_map_2.update_layout(mapbox_style="open-street-map")
fig_map_2.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.Div([
        html.Div([
            html.H3('Column 1'),
            dcc.Graph(id='g1', figure={'data': [{'y': [1, 2, 3]}]})
        ], className="two columns"),

        html.Div([
            html.H3('Column 2'),
            dcc.Graph(id='g2', figure={'data': [{'y': [1, 2, 3]}]})
        ], className="ten columns"),
    ], className="row")
])


#-----------------------------------Define Pies Here------------------------------------------------------

fig_pie = px.pie(collisions, values='number_of_persons_injured', 
             names='vehicle_type_code1', 
             title='Population of European continent',
              color_discrete_map={'Thur':'lightcyan',
                                 'Fri':'cyan',
                                 'Sat':'royalblue',
                                 'Sun':'darkblue'},
                hole=0.5
                )
fig_pie.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    font= {"color": "#ffc434"},
     title={
        'text': "Vehicles causing collisions",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
  )
fig_pie.update_layout(showlegend=True,)
fig_pie.update_traces(textposition='inside')
fig_pie.update_layout(uniformtext_minsize=12)




fig_pie2 = px.pie(collisions, values='number_of_persons_injured', 
             names='vehicle_type_code2', 
             title='Population of European continent',
             color_discrete_map={'Thur':'lightcyan',
                                 'Fri':'cyan',
                                 'Sat':'royalblue',
                                 'Sun':'darkblue'},
                hole=0.5
                )
fig_pie2.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
      font= {"color": "#ffc434"},
         title={
        'text': "Vehicles affected by the collisions",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
  )
fig_pie2.update_layout(showlegend=True)
fig_pie2.update_traces(textposition='inside')
fig_pie2.update_layout(uniformtext_minsize=12)




app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# styling the sidebar
sidebar = html.Div([   
        html.H2("InnoLab", className="display-4"),
        html.Hr(),
        html.P(
            "Collisions in New York", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Home", href="/", active="exact"),
                dbc.NavLink("NY Col Map", href="/page-1", active="exact"),
                dbc.NavLink("Bar Plot", href="/page-2", active="exact"),

            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)
content = html.Div(id="row", children=[], style=CONTENT_STYLE)

#Here Start the HTML Layout
app.layout = html.Div([
    dcc.Location(id="url"),
    sidebar,
    content
])

@app.callback(
    Output("row", "children"),
    [Input("url", "pathname")]
)
def render_page_content(pathname):
    
#---------------------------------------------Home Page----------------------------------------------------    
    
    if pathname == "/":
        return [
                #First Layer
                   html.H1('Newyork Collisions Dashboard',
                        style={'textAlign':'center','width': '100%','color':'#ffc434','marginBottom':'50px'}),
                   html.Div(children=[dcc.Graph(id='boxplot1', figure=fig_map_1),],
                        style={'display': 'inline-block','width': '100%','marginBottom':'50px'}),
                   html.Div(children=[
                    dcc.Graph(id='boxplot2', figure=fig_pie,style={'display': 'inline-block','width': '40%','marginLeft':'70px'}),
                    dcc.Graph(id='boxplot2', figure=fig_pie2,style={'display':'inline-block','width': '40%','marginLeft':'100px'}),],
                    style={'width': '100%', 'display': 'inline-block'}),     
                #Second Layer
                  html.Div(children=[dcc.Graph(id='boxplot3', figure=fig_box1),],
                    style={}),   
                  html.H3('In this app Statistical tables of traffic accidents in new york since 2016 is shown. OpenDash is used as web server. Tables and maps are taken from the Plolty library.',
                    style={'backgroundColor':'#201c1c','fontSize':'20px','color':'#ffc434','textAlign':'center','margin':'-47px','marginLeft':'300px','marginRight':'300px','marginBottom':'50px'}),
               ]
#---------------------------------------------Maps Page----------------------------------------------------    
                      
    elif pathname == "/page-1":
        return [
                   html.H1('Plotly Maps',
                            style={'textAlign':'center','width': '100%','color':'#ffc434'}),
                   html.Div(children=[
                        dcc.Graph(id='boxplot2', figure=fig_map_1,style={'display': 'inline-block','width': '40%','marginLeft':'70px'}),
                        dcc.Graph(id='boxplot2', figure=fig_map_2,style={'display':'inline-block','width': '40%','marginLeft':'100px'}),],)]
#---------------------------------------------Home Page----------------------------------------------------                                        
    elif pathname == "/page-2":
        return [
                     html.H1('Plotly Bars',
                        style={'textAlign':'center','width': '100%','color':'#ffc434','marginBottom':'200px'}),
             html.Div(children=[
                        dcc.Graph(id='boxplot3', figure=fig_box1),
                        dcc.Graph(id='boxplot3', figure=fig_box2)],),]
    
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )


if __name__=='__main__':
    app.run_server(debug=False, port=3000)




C:\Users\ahmet\AppData\Local\Temp/ipykernel_316/3412788668.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\ahmet\AppData\Local\Temp/ipykernel_316/3412788668.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:3000/

Dash is running on http://127.0.0.1:3000/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jan/2022 21:18:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 21:18:03] "GET /assets/style.css?m=1642047898.9519527 HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2022 21:18:03] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 21:18:03] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 21:18:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 21:18:03] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 21:18:03] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 21:18:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 21:19:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 21:19:06] "GET /page-2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 21:19:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 21:1